# OpenAI Cost Tracking

In [ ]:
import json
import requests
import datetime
from dateutil.relativedelta import relativedelta
import dateutil.parser as dparser

import os
import pandas as pd
import time

openai_org_id = os.getenv('OPENAI_ORG_ID')
openai_api_key = os.getenv('OPENAI_API_KEY')


files = []
[files.append(f) for f in os.listdir() if f.endswith('.json')]
files.sort()

fdates = {}

idate = datetime.date(2023, 7, 1)

for f in files:
    dates = []
    for i in f.split('_'):
        try:
            dparser.parse(i, fuzzy=True)
            
            if dparser.parse(i, fuzzy=True).date()>idate:
                
                first_day_of_month=dparser.parse(i, fuzzy=True).date()
                
            dates.append(dparser.parse(i, fuzzy=True)) 
            
        except ValueError:
            
            pass
        
    fdates[f] = dates
    
first_day_of_month += relativedelta(days=1)
current_day = datetime.date.today() #- datetime.timedelta(days = 1)#end date

headers = {
    "method": "GET",
    "authority": "api.openai.com",
    "scheme": "https",
    "path": f"/v1/organizations/{openai_org_id}/users",
    "authorization": f"Bearer {openai_api_key}",
}

starttime = time.monotonic()

users_response = requests.get(f"https://api.openai.com/v1/organizations/{openai_org_id}/users", headers=headers)
users = users_response.json()["members"]["data"]

df_costs = pd.DataFrame()

for user in users:
    
    id_of_user = user["user"]["id"]

    current_date = first_day_of_month
    
    while current_date < current_day and True:
#         print(f'{user} e {current_date}')
        time.sleep(15.0 - ((time.monotonic() - starttime) % 15.0))
        
        usage_headers = {
            "method": "GET",
            "authority": "api.openai.com",
            "authorization": f"Bearer {openai_api_key}",
            "openai-organization": openai_org_id,
        }
        
        usage_response = requests.get(f"https://api.openai.com/v1/usage?date={current_date}&user_public_id={id_of_user}", headers=usage_headers)
        user_data = usage_response.json()
     
        if len(user_data['data'])==0:
            
            current_date += relativedelta(days=1)
            
            continue
            
        else:
            
            data = user_data["data"]
            
            if len(user_data['whisper_api_data'])>0:
                
                wad = []
                for w in user_data['whisper_api_data']:
                    wd={}
                    wd['aggregation_timestamp'] = w['timestamp']
                    wd['n_requests'] = w['num_requests']
                    wd['operation'] = 'audio-whisper'
                    wd['snapshot_id'] = w['model_id']
                    wd['n_context'] = w['num_seconds']
                    wd['n_context_tokens_total'] = w['num_seconds']
                    wd['n_generated'] = 0
                    wd['n_generated_tokens_total'] = 0
                    wad = wad + [wd]
                data = data + wad
                
            else:
                
                pass
              
            df = pd.DataFrame(data)
            df['local_timestamp'] = df['aggregation_timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x))
            # converting to openai tz
            df['system_timestamp'] = df['local_timestamp'].dt.tz_localize('America/Sao_Paulo').dt.tz_convert("UTC")
            df['user'] = user["user"]["name"].lower().replace(" ", "_")
            df['email'] = user["user"]["email"]

            df_costs = pd.concat([df_costs,df])

            current_date += relativedelta(days=1)

# model costs

model_costs = {
    "gpt-3.5-turbo": {"context": 0.0015, "generated": 0.002},
    "gpt-3.5-turbo-0301": {"context": 0.0015, "generated": 0.002},
    "gpt-3.5-turbo-0613": {"context": 0.0015, "generated": 0.002},
    "gpt-3.5-turbo-16k": {"context": 0.003, "generated": 0.004},
    "gpt-3.5-turbo-16k-0613": {"context": 0.003, "generated": 0.004},
    "gpt-4": {"context": 0.03, "generated": 0.06},
    "gpt-4-0314": {"context": 0.03, "generated": 0.06},
    "gpt-4-0613": {"context": 0.03, "generated": 0.06},
    "gpt-4-32k": {"context": 0.06, "generated": 0.12},
    "gpt-4-32k-0314": {"context": 0.06, "generated": 0.12},
    "gpt-4-32k-0613": {"context": 0.06, "generated": 0.12},
    "text-embedding-ada-002-v2": {"context": 0.0001, "generated": 0},
    "text-davinci:003": {"context": 0.02, "generated": 0.02},
    "whisper-1": {"context": 0.1, "generated": 0}, # (0.006 / 60 sec) * 1000 - due to cost by 1000 tokens
}

mc = pd.DataFrame(model_costs)
mc = mc.T.reset_index()

# cost calculation

df_costs=df_costs.merge(mc, left_on='snapshot_id', right_on='index', how='left')
df_costs['context_costs']=(df_costs['n_context_tokens_total']/1000)*df_costs['context']
df_costs['generated_costs']=(df_costs['n_generated_tokens_total']/1000)*df_costs['generated']
df_costs['total_costs']=df_costs['context_costs']+df_costs['generated_costs']

# hist + current data

try:
    past_df = pd.read_json(files[len(files)-1])
    df_costs = pd.concat([df_costs, past_df])
except FileNotFoundError:
    pass
df_costs.reset_index(drop='level_0', inplace=True)

df_costs['local_timestamp'] = pd.to_datetime(df_costs['local_timestamp'])
df_costs['system_timestamp'] = pd.to_datetime(df_costs['system_timestamp'])

# saving a json file
    
with open(f"openai_costs_{idate}_to_{current_date - datetime.timedelta(days=1)}.json", "w") as f:
    json.dump(json.loads(df_costs.to_json(orient='columns', date_format='iso')), f)


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
def summary_costs(x):
    names = {
        'requests': x['n_requests'].sum()
        , 'context_tokens': x['n_context_tokens_total'].sum()
        , 'generated_tokens': x['n_generated_tokens_total'].sum()
        , 'context_costs': x['context_costs'].sum()
        , 'generated_costs': x['generated_costs'].sum()
        , 'total_costs': x['total_costs'].sum()
    }
    
    return pd.Series(names, index=['requests'
                                   , 'context_tokens'
                                   , 'generated_tokens'
                                   , 'context_costs'
                                   , 'generated_costs'
                                   , 'total_costs'
                                  ])

In [ ]:
daily_cost = df_costs.groupby([pd.Grouper(key='local_timestamp', freq='D')]).apply(summary_costs).reset_index()
daily_cost['month_cumulated'] = daily_cost.groupby(pd.Grouper(key='local_timestamp', freq='M'))['total_costs'].cumsum()
daily_cost['day'] = daily_cost['local_timestamp'].apply(lambda x: x.day)
daily_cost['month'] = daily_cost['local_timestamp'].apply(lambda x: x.month)
daily_plot = daily_cost.pivot_table(index='day', columns='month', values='month_cumulated', aggfunc='sum')
plt.figure(figsize=(10,10))
sns.lineplot(data=daily_plot, legend=True, sizes=(20, 2000))
plt.title('Cumulated spent by day - Current x Past Month')
plt.show()

In [ ]:
#costs by user

user_cost = df_costs.loc[df_costs['local_timestamp'].dt.month==datetime.date.today().month].groupby(['email'], group_keys=False).apply(summary_costs).sort_values('total_costs', ascending=False)[['total_costs']]
snap_cost = df_costs.loc[df_costs['local_timestamp'].dt.month==datetime.date.today().month].pivot_table(index='email', columns='snapshot_id', values='total_costs', aggfunc='sum')
user_snap_cost = user_cost.merge(snap_cost, left_index=True, right_index=True, how='left')
user_snap_cost

In [ ]:
plt.figure(figsize=(10,10))
user_snap_cost.drop('total_costs', axis=1).plot(kind='barh', stacked=True, color=['blue', 'red', 'purple', 'orange', 'yellow', 'skyblue', 'green'])
plt.gca().invert_yaxis()
plt.title('Total Spent by User/Model - Current Month')
plt.show()